<h1 align="center">Bolometric Corrections</h1>

### "Bolometric correction methods use the bolometric lightcurves of well-observed SNEe (by integrating the SED) to find correlations between an observable quantity such as color and the bolometric correction BC" Lusk et al $\href{https://arxiv.org/abs/1608.08631}{2016}$

Bolometric corrections are defined with the following two expressions

$$
BC_{\lambda} = m_{bol} - m_{\lambda}\\
$$
Where 
$$
BC_{\lambda} = \sum_{k=0}^{n} c_k (m_x - m_y)^k
$$

The bolometric corrections that we will use on KSP ZN-7090 will be the ones from the following two papers , Martinez et al. $\href{https://ui.adsabs.harvard.edu/abs/2022A%26A...660A..40M/abstract}{2022}$ and Layman et al. $\href{https://ui.adsabs.harvard.edu/abs/2014MNRAS.437.3848L/abstract}{2014}$, where they study a wide group of SNe measured in a variety of bands, which allowed them to construct integrate their SED over a range of wavelengths and obtain their bolometric light curves. After this they looked at the correlation beteen the colors of the SNe and the BC. Here they present their correction in seperate phases for the SNe. All the correction presented by both papers are polynomials of degree 2.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy import table

In [3]:
# Read data as a Panda data frame for easier handling
KSP_data = pd.read_csv("KSP-ZN7090_LC_20220309_correct.csv")

In [14]:
KSP_data

,date,date[MJD],date_err,telescope,band,M_0,M_0_err,M_0_lim,M_C,M_C_err,M_C_lim,M_E,M_E_err,M_E_lim,binned,binned_images
0,2020-10-06 19:08:45.000001,59128.797743,0.202431,A/S,V,NaN,NaN,21.839,NaN,NaN,21.839,NaN,NaN,21.772,Y,201006_1239.A/201006_2018.S/201006_2116.S/2010...
1,2020-10-06 19:10:45.000000,59128.799132,0.202431,A/S,I,NaN,NaN,21.944,NaN,NaN,21.944,NaN,NaN,21.902,Y,201006_1241.A/201006_2020.S/201006_2118.S/2010...
2,2020-10-06 21:16:40.000000,59128.886574,0.043056,S,B,23.035,0.506,22.247,23.031,1.240,22.035,22.940,1.240,21.944,Y,201006_2016.S/201006_2114.S/201006_2220.S
3,2020-10-07 21:18:00.000000,59129.887500,0.049653,S,B,25.710,4.268,22.597,26.831,5.908,22.385,26.740,5.908,22.294,Y,201007_2009.S/201007_2113.S/201007_2232.S
4,2020-10-07 21:20:00.000000,59129.888889,0.049653,S,V,NaN,NaN,22.232,NaN,NaN,22.232,NaN,NaN,22.165,Y,201007_2011.S/201007_2115.S/201007_2234.S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2020-12-09 10:42:00.000000,59192.445833,0.000000,A,V,20.713,0.099,21.712,20.713,0.099,21.712,20.646,0.099,21.645,N,NaN
130,2020-12-09 10:44:00.000000,59192.447222,0.000000,A,I,20.305,0.126,21.045,20.305,0.126,21.045,20.263,0.126,21.003,N,NaN
131,2020-12-10 01:16:00.000000,59193.052778,0.000000,C,B,21.277,0.087,22.449,21.180,0.124,22.237,21.089,0.124,22.146,N,NaN
132,2020-12-10 01:18:00.000000,59193.054167,0.000000,C,V,20.757,0.076,22.085,20.757,0.076,22.085,20.690,0.076,22.018,N,NaN


In [44]:
# Store dates in local variable
dates = np.array(KSP_data['date'])

# Look at the bands of each individual measurement
bands = KSP_data["band"]
loc_B = np.where(bands=="B")
loc_V = np.where(bands=="V")
loc_I = np.where(bands=='I')

# Index the corrected magnitude arrays on the bands of interest
all_mags = np.array(KSP_data["M_C"])
all_mag_err = np.array(KSP_data["M_C_err"])
B_mags = all_mags[loc_B]
V_mags = all_mags[loc_V]
I_mags = all_mags[loc_I]

# Index dates for respective bands
B_dates = dates[loc_B]
V_dates = dates[loc_V]
I_dates = dates[loc_I]

In [55]:
# Convert dates to MJD 
from astropy.time import Time
print(dates)
JD_Dates = Time(dates, format='isot')

['2020-10-06 19:08:45.000001' '2020-10-06 19:10:45.000000'
 '2020-10-06 21:16:40.000000' '2020-10-07 21:18:00.000000'
 '2020-10-07 21:20:00.000000' '2020-10-07 21:22:00.000000'
 '2020-10-09 13:15:20.000000' '2020-10-09 13:17:20.000000'
 '2020-10-09 13:19:19.999999' '2020-10-12 14:44:00.000000'
 '2020-10-12 14:47:00.000000' '2020-10-12 20:58:40.000000'
 '2020-10-12 21:00:40.000000' '2020-10-12 21:02:40.000000'
 '2020-10-13 12:06:00.000000' '2020-10-13 12:08:00.000000'
 '2020-10-13 12:10:30.000000' '2020-10-13 21:25:45.000000'
 '2020-10-13 21:28:00.000000' '2020-10-13 21:30:00.000000'
 '2020-10-15 11:19:00.000000' '2020-10-15 11:21:00.000000'
 '2020-10-15 11:23:00.000000' '2020-10-15 12:20:00.000000'
 '2020-10-15 12:23:00.000000' '2020-10-15 12:25:00.000000'
 '2020-10-15 13:44:00.000000' '2020-10-15 13:46:00.000000'
 '2020-10-15 13:48:00.000000' '2020-10-15 19:36:00.000000'
 '2020-10-15 19:38:00.000000' '2020-10-15 19:41:00.000000'
 '2020-10-15 20:37:00.000000' '2020-10-15 20:39:00.00000

ValueError: Input values did not match the format class isot:
TypeError: Input values for isot class must be strings